# MusicBrainz place geocoder

To see this analysis live, check out my article ["Analyzing Last.fm Listening History"](http://geoffboeing.com/2016/05/analyzing-lastfm-history/)

This notebook loads a set of artists from musicbrainz, created by the [musicbrainz_downloader](musicbrainz_downloader.ipynb). Then it takes each's place name (ie, either where they're from or where they're most associated with - as determined in other notebook), and geocodes that place name to lat long. Then it maps the artists.

Nominatim API documentation: https://wiki.openstreetmap.org/wiki/Nominatim

Sample Nominatim query: https://nominatim.openstreetmap.org/search?format=json&q=brixton,london,england

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time, requests, re
from mpl_toolkits.basemap import Basemap
from geopy.distance import great_circle

%matplotlib inline
pause = 0.75

## Define geocoding functions

Nominatim and Google APIs

In [ ]:
def geocode_nominatim(address):
    time.sleep(pause)
    url = 'https://nominatim.openstreetmap.org/search?format=json&q={}'
    request = url.format(address)
    response = requests.get(request)
    data = response.json()
    if len(data) > 0:
        return '{},{}'.format(data[0]['lat'], data[0]['lon'])

In [ ]:
def geocode_google(address):
    time.sleep(pause)
    url = 'http://maps.googleapis.com/maps/api/geocode/json?sensor=false&address={}'
    request = url.format(address)
    response = requests.get(request)
    data = response.json()
    if len(data['results']) > 0:
        latitude = data['results'][0]['geometry']['location']['lat']
        longitude = data['results'][0]['geometry']['location']['lng']
        return '{},{}'.format(latitude, longitude)

In [ ]:
# to improve geocoding accuracy, remove anything in parentheses or square brackets
# example: turn 'Tarlac, Luzon (Region III), Philippines' into 'Tarlac, Luzon, Philippines'
regex = re.compile('\\(.*\\)|\\[.*\\]')
def clean_place_full(place_full):
    if isinstance(place_full, str):
        return regex.sub('', place_full).replace(' ,', ',').replace('  ', ' ')

## Test it

In [ ]:
address = 'Brixton, London, England, United Kingdom'
latlng_google = geocode_google(address)
latlng_nominatim = geocode_nominatim(address)

In [ ]:
print('{} google'.format(latlng_google))
print('{} nominatim'.format(latlng_nominatim))
print('{:.1f} meters apart'.format(great_circle(latlng_google, latlng_nominatim).m))

## Run it

In [ ]:
# load the dataset
artists = pd.read_csv('data/mb.csv', encoding='utf-8')
print('{:,} total artists'.format(len(artists)))

# clean place_full to remove anything in parentheses or brackets and change empty strings to nulls
artists['place_full'] = artists['place_full'].map(clean_place_full)
artists.loc[artists['place_full']=='', 'place_full'] = None

# drop nulls and get the unique set of places
addresses = pd.Series(artists['place_full'].dropna().sort_values().unique())
print('{:,} unique places'.format(len(addresses)))

In [ ]:
def get_country_if_more_detail(address):
    tokens = address.split(',')
    if len(tokens) > 1:
        return tokens[-1].strip()

# if a place contains only country name, check if that country name exists with more detail elsewhere in the list of places
# countries_with_more_detail is a list of all the countries that appear at end of comma-separated address strings
countries_with_more_detail = pd.Series(addresses.map(get_country_if_more_detail).dropna().sort_values().unique())
print('{:,} countries with more detail'.format(len(countries_with_more_detail)))

# if so, discard the instance that is country name only - this country is represented elsewhere in list with finer grain info
# e.g., keep 'estonia' if there is no 'talinn, estonia' (or other city) elsewhere in list, 
# but discard 'russia' if 'moscow, russia' exists elsewhere in the list
addresses_to_geocode = addresses[~addresses.isin(countries_with_more_detail)]
print('{:,} unique addresses to geocode'.format(len(addresses_to_geocode)))

In [ ]:
# geocode (with nominatim) each retained address (ie, full place name string)
start_time = time.time()

latlng_dict = {}
for address, n in zip(addresses_to_geocode, range(len(addresses_to_geocode))):
    if n % 10 == 0: print(n, end=' ')
    latlng_dict[address] = geocode_nominatim(address)

finish_time = time.time()

In [ ]:
print('nominatim geocoded {:,} addresses in {:,} seconds'.format(len(addresses_to_geocode), int(finish_time-start_time)))
print('received {:,} non-null lat-longs'.format(len([key for key in latlng_dict if latlng_dict[key] is not None])))

In [ ]:
# which addresses failed to geocode successfully?
addresses_to_geocode = [ key for key in latlng_dict if latlng_dict[key] is None ]

In [ ]:
# now geocode (with google) each address that failed with nominatim
if len(addresses_to_geocode) < 2500: #daily google request limit
    start_time = time.time()
    for address, n in zip(addresses_to_geocode, range(len(addresses_to_geocode))):
        if n % 10 == 0: print(n, end=' ')
        latlng_dict[address] = geocode_google(address)
    finish_time = time.time()

In [ ]:
print('google geocoded {:,} addresses in {:,} seconds'.format(len(addresses_to_geocode), int(finish_time-start_time)))
print('received {:,} non-null lat-longs'.format(len([key for key in latlng_dict if latlng_dict[key] is not None])))

In [ ]:
# for each artist, if their place appears in the geocoded dict, pull the latlng value from dict into new df column
def get_latlng_by_address(address):
    try:
        return latlng_dict[address]
    except:
        return None

artists['place_latlng'] = artists['place_full'].map(get_latlng_by_address)
artists[['name', 'place_full', 'place_latlng']].sort_values(by='place_full').head()

In [ ]:
artists.to_csv('data/mb_geocoded.csv', index=False, encoding='utf-8')

## Map it

In [ ]:
# get discrete vectors of lats and lons, for easy x-y scatter-plotting
lats = artists['place_latlng'].dropna().map(lambda x: float(x.split(',')[0]))
lons = artists['place_latlng'].dropna().map(lambda x: float(x.split(',')[1]))

In [ ]:
# define map colors
land_color = '#f5f5f3'
water_color = '#cdd2d4'
coastline_color = '#f5f5f3'
border_color = '#bbbbbb'
meridian_color = '#f5f5f3'
marker_fill_color = '#cc3300'
marker_edge_color = 'None'

In [ ]:
# create the plot
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111, axisbg='#ffffff', frame_on=False)
ax.set_title('Last.fm Artist Origins', fontsize=24, color='#333333')

# draw the basemap and its features
m = Basemap(projection='kav7', lon_0=0, resolution='l', area_thresh=10000)
m.drawmapboundary(color=border_color, fill_color=water_color)
m.drawcoastlines(color=coastline_color)
m.drawcountries(color=border_color)
m.fillcontinents(color=land_color, lake_color=water_color)
m.drawparallels(np.arange(-90., 120., 30.), color=meridian_color)
m.drawmeridians(np.arange(0., 420., 60.), color=meridian_color)

# project our points from each dataset then concatenate and scatter plot them
x, y = m(lons.values, lats.values)
m.scatter(x, y, s=8, color=marker_fill_color, edgecolor=marker_edge_color, alpha=1, zorder=3)

# show the map
plt.savefig('images/lastfm_artists_origins_map.png', dpi=96, bbox_inches='tight', pad_inches=0.2)
plt.show()